In [1]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

../scripts/functions.py:101: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [2]:
length=[4, 4]
V = 1.0
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))
op_transition1 = f.dimer_flip1(length = np.array(length))


hex_ = nk.machine.new_hex(np.array(length))

ma = nk.machine.RbmDimer(hi, hex_, alpha = 2, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

2


In [3]:
ma.load('../save/ma/h=1.0V=0.9l=[4, 4]')

In [4]:
ma._ws[:] = 0

In [5]:
ma._set_bare_parameters(
    ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
)

In [17]:
# N = 1000000
# temp = np.random.choice([-1,1],hex_.size*N).reshape(N, hex_.size)
# ind = hex_.is_dimer_basis(temp)

In [31]:
a = np.load('../save/dynamics/h=1.0V=l=[4, 4]/P_n=1.0e+05_0.npy')
t_list = np.arange(0,30,0.1)

In [13]:
import pickle
with open("yo", "wb") as fp:   #Pickling
    pickle.dump([a,t_list], fp)

In [14]:
with open("yo", "rb") as fp:   # Unpickling
    b = pickle.load(fp)

In [26]:
a_ = np.array([a,t_list],dtype=object)
np.save('yo',a_)

ValueError: could not broadcast input array from shape (300,99984,32) into shape (300)

In [4]:

# create save folder.


n_jobs = 12

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))




sweep_size = 1000
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition, length = length, n_chains=1, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sa_mul.reset()
sa_mul.generate_samples(1000) # discard the begginings of metropolis sampling.



number of core : 12


array([[[ 1.,  1.,  1., ...,  1., -1., -1.]],

       [[-1., -1.,  1., ..., -1., -1.,  1.]],

       [[ 1., -1.,  1., ..., -1., -1., -1.]],

       ...,

       [[-1., -1.,  1., ..., -1., -1., -1.]],

       [[ 1.,  1.,  1., ..., -1., -1., -1.]],

       [[ 1., -1., -1., ..., -1., -1., -1.]]])

In [5]:
n_samples_ = 50000
samples_state = sa_mul.generate_samples(int(n_samples_ / 1))
samples_state = samples_state.reshape(-1, ma.hilbert.size)
print('get samples')

get samples


In [6]:
import scripts.dynamics as Dynamics
importlib.reload(Dynamics)

<module 'scripts.dynamics' from '../scripts/dynamics.py'>

In [7]:
d = Dynamics.new_dynamics(op, ma)
t_list = np.linspace(0,30,100)
%time P_ = d.multiprocess(samples_state, 300, 12) 

CPU times: user 205 ms, sys: 675 ms, total: 879 ms
Wall time: 9.63 s


In [8]:
P = P_[0]
T = P_[1]

# t_list = np.linspace(0,50,100)
t_list = np.arange(0,20,0.1)
P_list = f.process_P(P, T, t_list)

In [9]:
edge_corr1 = np.array([ 0.5     ,  0.      ])
edge_corr2 = np.array([ 0.25    , -0.433013])
edge_corr2 = np.array([ 0.75    , -0.433013])

In [11]:
edge_corr1

array([0.5, 0. ])

In [10]:
hex_.edge_coor_to_lattice(edge_corr1)[0]

array([4, 8])

In [19]:
hex_.a1

array([1., 0.])

In [21]:
hex_.a2[None,:] * 1/2

array([[ 0.25      , -0.43301269]])

In [80]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))


base_edge = hex_.get_edge_coor(np.array([4,8]))
x = np.arange(length[0]*2)
y = np.arange(length[1]*2)

xx, yy = np.meshgrid(x,y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)
edge_coor_array = hex_.ProcessPeriodic(xx[:,None] * hex_.a1[None,:] * 1/2 + yy[:,None]  * hex_.a2[None,:] * 1/2 + base_edge)
edges = hex_.edge_coor_to_lattice(edge_coor_array)
colors = hex_.get_edge_color(edges)

operators = f.return_spin_corr(hi, edges, colors)

In [23]:
edge_coors = np.array([[ 0.5     ,  0.      ],
                       [ 0.25    , -0.433013],
                       [ 0.75    , -0.433013]])

a = np.tan((1/6)*np.pi) * 1/2
edge_coors_prime = np.array([
    hex_.a((1/2)*np.pi) * a,
    hex_.a(np.pi*(2/3)+(1/2)*np.pi) * a,
    hex_.a(np.pi*(4/3)+(1/2)*np.pi) * a,
])
edge_coors_prime = np.round(edge_coors_prime,10)

In [24]:
edge_coors_prime

array([[ 0.        ,  0.28867513],
       [-0.25      , -0.14433757],
       [ 0.25      , -0.14433757]])

In [25]:
K = np.zeros((2*length[0],2*length[1],2))

In [26]:
A = np.array([
    hex_.a1,
    hex_.a2
])

In [27]:
A

array([[ 1.        ,  0.        ],
       [ 0.5       , -0.86602539]])

In [28]:
[b1, b2] = np.linalg.inv(A.T)

In [29]:
b1 

array([1.        , 0.57735028])

In [30]:
b1 @ hex_.a1

1.0

In [31]:
K = np.zeros((2*length[0],2*length[1],2))
for l1 in range(K.shape[0]):
    for l2 in range(K.shape[1]):
        K[l1,l2] = hex_.b1 * 2*np.pi*l1/K.shape[0] + hex_.b2 * 2*np.pi*l2/K.shape[1]

In [32]:
edge_coors_prime[0]

array([0.        , 0.28867513])

In [33]:
uni = np.exp(1j*np.einsum('ijk,lk->lij',K,edge_coors_prime))

In [23]:
K.shape

(8, 8, 2)

In [50]:
2*length[0]

8

In [34]:
num_samples = (P_list[:,:,0]!=0).sum(axis=1)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
sections1 = np.arange(P_list.shape[1])
sections2 = np.zeros(P_list_.shape[0])

_, mels1 = operators[0].get_conn_flattened(P_list[0,:,:], sections1)
sub1 = mels1.mean().real
mels1 = mels1.real


dimer_corr = np.zeros((2*length[0],2*length[1],t_list.shape[0]))
dimer_std = np.zeros((2*length[0],2*length[1],t_list.shape[0]))


for l2 in range(2*length[1]):
    for l1 in range(2*length[0]):
#     l1 = 1
#     l2 = 0
        print(l1,l2)
        if operators[l1 + l2 * 2*length[0]]:
            sub2 = operators[l1 + l2 * 2*length[0]].get_conn_flattened(P_list[0,:,:], sections1)[1].mean().real

            _, mels2_ = operators[l1 + l2 * 2*length[0]].get_conn_flattened(P_list_, sections2)
            mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
            dimer_corr[l1,l2] = (mels2 * mels1).sum(axis=1)/num_samples 
            dimer_std[l1,l2] = (mels2 * mels1).std(axis=1)
        else:
            dimer_corr[l1,l2] = 0
            dimer_std[l1,l2] = 0

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
0 2
1 2
2 2
3 2
4 2
5 2
6 2
7 2
0 3
1 3
2 3
3 3
4 3
5 3
6 3
7 3
0 4
1 4
2 4
3 4
4 4
5 4
6 4
7 4
0 5
1 5
2 5
3 5
4 5
5 5
6 5
7 5
0 6
1 6
2 6
3 6
4 6
5 6
6 6
7 6
0 7
1 7
2 7
3 7
4 7
5 7
6 7
7 7


In [40]:
tau_list = []
N = 2000
corr1 = []
corr2 = np.zeros((81,N),dtype=np.float)
n = -1
import re
with open('RN4_4V0.9st.dat') as fe:
    for line in fe:
        
        if re.findall(r'#\d', line):
            i = 0
            n += 1
        else:
            try:
                match = re.findall(r'[\d,\.]+', line)
                tau_list.append(float(match[1]))
                corr2[n,i] = float(match[3])
                i+=1
            except:
                pass

In [79]:
np.abs(corr2[3][:200][None,None,:] - dimer_corr).mean(axis=-1).reshape(-1,order='F').argsort()

array([ 2,  6, 57, 15, 11, 56,  8, 25, 60, 61, 12, 44, 40, 43, 24, 47, 28,
        9, 29, 58, 63, 14, 27, 45, 30, 59, 26, 41, 46, 13, 62, 31, 10, 42,
       34, 38, 22, 54, 18, 50, 48, 20, 52, 32, 16, 36,  4,  0,  1,  3,  5,
        7, 33, 55, 21, 53, 51, 37, 49, 39, 23, 17, 19, 35])

In [78]:
(corr2[3][:200][None,None,:] - dimer_corr).mean(axis=-1).reshape(-1,order='F')

array([-0.05634956,  0.42758727,  0.00322997,  0.42758727, -0.0441282 ,
        0.42758727,  0.0046754 ,  0.42758727, -0.00115713, -0.00917361,
       -0.01261676, -0.00640277, -0.00570326, -0.0147057 , -0.01300102,
       -0.00211408, -0.02437084,  0.42758727, -0.01894938,  0.42758727,
       -0.02238292,  0.42758727, -0.02108872,  0.42758727, -0.00573926,
       -0.00756955, -0.01115713, -0.01382396, -0.00062284, -0.0066182 ,
       -0.0128662 , -0.01682243, -0.0241016 ,  0.42758727, -0.01379675,
        0.42758727, -0.02562104,  0.42758727, -0.01401078,  0.42758727,
       -0.0072511 , -0.01454327, -0.01759956, -0.00695526, -0.00468229,
       -0.01462088, -0.01405399, -0.00369133, -0.02069845,  0.42758727,
       -0.02181723,  0.42758727, -0.02030079,  0.42758727, -0.02049422,
        0.42758727,  0.00094381, -0.0047047 , -0.01026439, -0.01439945,
       -0.0041022 , -0.00484732, -0.01371034, -0.01257396])

In [69]:
dimer_corr[2,0] - corr2[3][:200]

array([-3.12049928e-03, -7.45742143e-03, -7.81139807e-03, -8.61284048e-03,
       -5.28456942e-03, -5.25228981e-03, -5.37689334e-03, -5.76945519e-03,
       -6.41355522e-03, -6.35414781e-03, -5.81071617e-03, -6.17250941e-03,
       -6.47478285e-03, -3.91671614e-03, -3.42146716e-03, -2.31271109e-03,
       -2.63592153e-03, -3.03660050e-03, -1.44038885e-03, -3.69423600e-03,
       -2.56289996e-03, -3.33295532e-03, -3.56950088e-03, -3.31776248e-03,
       -3.62284320e-03, -2.28904266e-03, -3.20892772e-04, -1.40308406e-03,
       -3.09906977e-03,  3.88478667e-04,  1.97505845e-03,  4.53797591e-03,
        2.03331111e-03, -1.40837668e-04, -5.46831223e-04, -6.27328841e-04,
       -3.34508206e-03, -3.50139631e-03, -3.09859879e-03, -1.61849225e-03,
       -5.43086833e-04,  8.86070562e-04,  2.58739609e-03,  1.55892937e-03,
        9.59780256e-04, -2.13183559e-03,  1.12425279e-03, -1.54870848e-04,
        2.43102647e-03,  3.03995871e-03, -5.29421869e-04, -2.15707932e-03,
       -3.61373556e-06,  

In [35]:
tau_list = []
N = 2000
corr1 = []
corr2 = np.zeros((81,N),dtype=np.complex128)
n = -1
import re
with open('K4_4V1.0st.dat') as fe:
    for line in fe:
        
        if re.findall(r'#\d', line):
            i = 0
            n += 1
        else:
            try:
                match = re.findall(r'[\d,\.]+', line)
                tau_list.append(float(match[1]))
                corr1.append(float(match[2]))
                corr2[n,i] = float(match[2]) + 1j*float(match[3])
                i+=1
            except:
                pass
            

In [39]:
t_list

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9, 10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9,
       11. , 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. ,
       12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1,
       13.2, 13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14. , 14.1, 14.2,
       14.3, 14.4, 14.5, 14.6, 14.7, 14.8, 14.9, 15

In [12]:
corr2[1]

array([0.02205296+0.01001865j, 0.01875094+0.01129549j,
       0.01601842+0.01156013j, ..., 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ])

In [68]:
fft_dimer_corr_prime - corr2[1]

array([[[ 6.41088174e+01+0.00000000e+00j,
          6.41088174e+01+0.00000000e+00j,
          6.41088174e+01+0.00000000e+00j, ...,
          6.41088174e+01+0.00000000e+00j,
          6.41088174e+01+0.00000000e+00j,
          6.41088174e+01+0.00000000e+00j],
        [ 8.10513451e-02+8.16017401e-02j,
          7.09706089e-02+6.75812434e-02j,
          5.71711079e-02+5.58663197e-02j, ...,
         -2.78779989e-04-6.12363881e-04j,
          9.84691330e-04-7.46648107e-04j,
          1.06420122e-03-6.65821767e-04j],
        [ 1.49657447e+00+4.01005927e-01j,
          1.34014673e+00+3.59091240e-01j,
          1.20976452e+00+3.24155432e-01j, ...,
         -3.86432159e-04-1.03544187e-04j,
          1.54572864e-03+4.14176748e-04j,
          1.77758793e-02+4.76303260e-03j],
        ...,
        [ 1.48309239e+00-1.09268509e-01j,
          1.23831994e+00-7.38834879e-02j,
          1.01975090e+00-6.75432109e-02j, ...,
         -1.02687688e-02+2.24319638e-03j,
         -9.05964646e-03+8.45728754e-03j

In [6]:
import pickle
with open('/home/keisuke/Documents/dimermaster/save/corr/h=1.0V=1.0l=[4, 4]/n=1.0e+05','rb') as fp:
    [fft_dimer_corr_prime, t_list, num_samples] = pickle.load(fp)

In [8]:
fft_dimer_corr_prime /= 64*2

In [37]:
corr2[1]

array([0.02205296+0.01001865j, 0.01875094+0.01129549j,
       0.01601842+0.01156013j, ..., 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ])

In [38]:
fft_dimer_corr_prime[0,1,:]

array([ 6.33213633e-04+6.37513595e-04j,  5.54457882e-04+5.27978464e-04j,
        4.46649281e-04+4.36455623e-04j,  3.74663997e-04+3.63840674e-04j,
        3.03567010e-04+3.09556971e-04j,  2.60809011e-04+2.66974483e-04j,
        2.10634693e-04+2.26089048e-04j,  1.57099475e-04+1.96404302e-04j,
        1.25699276e-04+1.77254209e-04j,  1.12148854e-04+1.57455846e-04j,
        9.95927854e-05+1.37463582e-04j,  9.47376444e-05+1.12288806e-04j,
        1.10165978e-04+9.78546576e-05j,  8.96645063e-05+8.97837436e-05j,
        7.08354584e-05+7.54618102e-05j,  7.64005194e-05+6.94734130e-05j,
        6.42691612e-05+6.30041517e-05j,  9.48382490e-05+5.26371036e-05j,
        7.09661667e-05+4.98940494e-05j,  6.48933038e-05+4.86198207e-05j,
        3.78528410e-05+4.27362413e-05j,  3.46178503e-05+4.04114623e-05j,
        2.13831400e-05+3.54709585e-05j,  2.77003357e-05+3.22550111e-05j,
        4.28213818e-05+2.90737732e-05j, -2.81113831e-06+2.46901837e-05j,
       -2.45452405e-05+2.12291871e-05j, -2.94688503

In [25]:
fft_dimer_corr_prime[0,1,:] - corr2[1,:][:200]

array([-2.14197469e-02-9.38113652e-03j, -1.81964785e-02-1.07675108e-02j,
       -1.55717703e-02-1.11236737e-02j, -1.34014821e-02-1.08837038e-02j,
       -1.16292045e-02-1.03108319e-02j, -1.01456184e-02-9.57139250e-03j,
       -8.91968221e-03-8.77225341e-03j, -7.89513255e-03-7.96181172e-03j,
       -7.00645758e-03-7.17423794e-03j, -6.22747065e-03-6.43888790e-03j,
       -5.55178303e-03-5.76387602e-03j, -4.95482743e-03-5.15681377e-03j,
       -4.41014334e-03-4.60080679e-03j, -3.96301221e-03-4.09718203e-03j,
       -3.56709001e-03-3.65439666e-03j, -3.19255558e-03-3.25322102e-03j,
       -2.87564633e-03-2.89772260e-03j, -2.55107175e-03-2.58670342e-03j,
       -2.31183033e-03-2.30429274e-03j, -2.08213541e-03-2.05263176e-03j,
       -1.89769030e-03-1.83414727e-03j, -1.71105512e-03-1.63738385e-03j,
       -1.55369914e-03-1.46557881e-03j, -1.39401436e-03-1.31178192e-03j,
       -1.24093140e-03-1.17537311e-03j, -1.16239636e-03-1.05555096e-03j,
       -1.07232519e-03-9.48395101e-04j, -9.76530559

In [23]:
t_list

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9, 10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9,
       11. , 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. ,
       12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1,
       13.2, 13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14. , 14.1, 14.2,
       14.3, 14.4, 14.5, 14.6, 14.7, 14.8, 14.9, 15

In [24]:
np.unravel_index(np.argmin(B),B.shape)

(7, 1)

In [147]:
np.unravel_index(np.argmin(A, axis=None), A.shape)

(7, 7)

In [152]:
np.where((corr2[1] - corr2[2]) != 0)

(array([ 696,  856, 1020]),)

In [145]:
dimer_corr[:,1,0]

array([0.33017283, 0.33421347, 0.45667307, 0.41334614, 0.41466635,
       0.45383261, 0.32621219, 0.3337734 ])

In [163]:
np.where(np.mean(corr2 - corr2[14],axis=1).round(5)==0)

(array([14, 22, 26, 31]),)

In [107]:
np.abs(dimer_corr[:,1] - corr2[4][:200]).mean(axis=1)

array([0.03491236, 0.44306074, 0.02083999, 0.44306074, 0.01132994,
       0.00236936, 0.00682221, 0.01105662])

In [16]:
corr2[5]

array([0.45833333, 0.45814134, 0.45771851, ..., 0.44444444, 0.44444444,
       0.44444444])

In [92]:
d = f._dynamics(
    op._local_states,
    op._basis,
    op._constant,
    op._diag_mels,
    op._n_conns,
    op._mels,
    op._x_prime,
    op._acting_on,
    op._acting_size
)

In [94]:
P = d.dynamics(samples_state.astype(np.int8), t_list, 0)

In [95]:
P_list = P.transpose(1,0,2)

In [52]:
t_list_ = t_list.reshape(-1,1)
n = 1
index = np.logical_and(T[n] <= t_list_,  t_list_ < T[n+10])

In [23]:
import scipy.fft as fft
fft_dimer_corr = np.empty_like(dimer_corr)
for t in range(t_list.shape[0]):
    fft_dimer_corr[:,:,t] = np.real(fft.fft2(dimer_corr[:,:,t]))

In [84]:
from plotly import graph_objects as go


In [86]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = t_list,
        y = dimer_corr[0, 0],
    )
)

fig.add_trace(
    go.Scatter(
        x = tau_list,
        y = corr2,
    )
)
fig.show()